# Pricing Forwards & Futures

Firstly, it'll probably be helpful for future me to define both forwards and futures. "All About Derivatives" defines
them as follows:

> A forward is a contract wherein a buyer agrees to purchase the underlier from the seller at a specified price on a specified future date

> A futures is a standardized forward contract executed at an exchange

Now we've defined them, what do we need to know about them to price them? We need to use and derive the following values:

* *Delivery Price*: the price the long party commits to buy and the short party commits to sell.
* *Forward/futures Price*: the delivery price for a new, zero-value forward contract.
* *Value*: the value is a measure of how much worse or better the parties are for having entered into the agreement.
* *Future Value (FV)*: the future value, calculated from the known value today.
* *Present Value (PV)*: the present value, calculated from a given future value. Adjusting backwards is known as *discounting*.
* *Spot Price (S)*: the current market price at which the asset is bought or sold, it changes over the course of the contract.
* *Carry*: the cost of maintaining a forward position over time.
* *Storage Cost*: a fixed cost of maintaining the underlying asset.
* *Time Period (t)*: the period which the contract covers.
* *Risk Free Interest Rate (r)*: the rate of return with no hypothetical loss, over time e.g. Swiss Bonds

## Forward Price Formulas

### Basic Forward

A basic forward is one where there is no storage cost, and generates no income. The only cost of carry is interest.  The future price, F is equal to the future value.

$$F = Se^{rt}$$

In [1]:
import math

def F_basic(S, r, t):
    return S * math.exp(r * t)

Let's try getting F for some example values. Lets say we're buying 100 shares of a stock in 6 months. The stock price is currently $15.50. The risk-free interest rate is 2\%. The future value is therefore:

In [2]:
S = 1550 # in pence
r = 0.02
t = 0.5 # in years
F_basic(S, r, t)

1565.5777589804604

Which is correct, so that's nice.

### Forward with Storage

The only difference here is that the forward price is now equal to the spot (S), *plus* the present value of storage (U). U is a fixed amount here.

In [3]:
def F_fixed_storage(S, r, t, U):
    return (S + U) * math.exp(r * t)

As an example, let's say one party wants to buy 5000 barrels of oil 1 year forward. The oils spot price is `$`32 a barrel.
The cost is `$`1.50 a barrel per quarter, payable at the beginning of each quarter. The risk free interest rate is 3\%. First we have to calculate U. This isn't as simple as it seems (to me). This is because we have to take the risk free interest rate into account.

In [4]:
cpq = 1.50 * 5000
r = 0.03
pv_s = cpq * (1 + math.exp(-r * 0.25) + math.exp(-r * 0.5) + math.exp(-r * 0.75))
# notice the present value of the storage is cheaper over time, due to interest
pv_s

29665.43423711653

Now to calculate the future price (the fair market delivery price):

In [5]:
F_fixed_storage(S=32*5000, r=0.03, t=1, U=pv_s)

195441.60664389934

What about when the storage price *is* proportional to the spot price? We already take into account one thing like this, the interest rate. This makes working it out simple:

In [6]:
def F_prop_storage(S, r, t, u):
    return S * math.exp((r + u) * t)

Let's say we want to buy 1000 bushels of apples from an apple growe in 3 months. Stored apples lose 2% of their value due to spoilage. Apples sell for $6 a bushel. The risk free interest rate is 4%.

In [7]:
F_prop_storage(S=1000*6, r=0.04, t=0.25, u=0.02)

6090.678387694314

### Forward with Income

This is the inverse of storage, instead of a cost per month, there is instead a benefit. This means we can use our existing formulas, with values being negatives. 

As an example with a fixed income, let's say we want to buy 400 shares of stock in 1 year. The stock is currently trading at `$`36 The company pays a quarterly dividend, expected to be `$`0.25 per share. The risk free interest rate is 2%.

In [13]:
div_val = (0.25 * 400) * (math.exp(-0.02 * 0.25) + math.exp(-0.02 * 0.5) + math.exp(-0.02 * 0.75) + math.exp(-0.02))
F_fixed_storage(S=400*36, r=0.02, t=1, U=-div_val)

14287.881721129355

Sometimes the income will be proportional to the spot price. As an example let's say we agree to buy 200 shares of a stock index in 1 year. The index is trading at $54 a share and it pays an annual dividend equal to 0.5% of its share price. The risk free interest rate is 3%.

In [14]:
F_prop_storage(S=200*54, r=0.03, t=1, u=-0.005)

11073.403301663831

### Forward Price of Currency

When the underlier is a foreign currency we have to consider not only the domestic interest rate but also the risk free foreign rate. The risk free foreign rate represents income to the short party and is just proportional income.

As an example, let's say we want to buy 5000 krinkets from a currency dealer in 9 months. Krinkets are currently trading for $0.35. The risk free interest rate in Krinketville is 2% and the risk free interest rate in the US is 6%. 

In [15]:
F_prop_storage(S=5000*0.35, r=0.06, t=0.75, u=-0.02)

1803.2954344186546

## Valuing an Existing Forward or Futures Position

At the time of inception the value of a forward price and a futures price are the same thing. After inceptionn, their values must be calculated differently because of the daily setttlements of futures contracts.

### Forward Contract Value

The current value of an existing forward contract is the difference between its delivery price and the delivery price of a theoretical new contract at this time (i.e. the current forward price).

$$ Forward Value_{Long} = PV(F - K) = (F - K)e^{-rt} $$

$$ Forward Value_{Short} = PV(K - F) = (K - F)e^{-rt} $$

The reason we need to discount is because the difference between the forward price F and our current contract price K is a future value and we need the present value.

In [16]:
def fcv(F, K, r, t, pos):
    if pos == 'long':
        return (F - K) * math.exp(-r * t)
    if pos == 'short':
        return (K - F) * math.exp(-r * t)

As an example, let's say we bought a 6 month forward contract to buy 100 shares at `$`15.66 a share when the spot price was `$`15.50. 3 months have gone by and the share price is now `$`15.80. The risk free interest rate r, is 3% and hasn't changed. 

Firstly we need to calculate the current forward price and our current forward value:

In [21]:
F = F_basic(S=15.80*100, r=0.03, t=0.25)
print(F)
K = F_basic(S=15.50*100, r=0.03, t=0.5)
print(K)

1591.8945488023635
1573.4252501543644


In [23]:
fcv(F=F, K=K, r=0.03, t=0.25, pos='long')

18.33129706097223

### Futures Contract Value

When calculating the current contract value, there are differences between forwards and futures. This is because the payoff from a futures contract is realized at the end of every day. This means that the future's contract value is the difference between the current futures price and the futures price at the end of the previously trading day when everyone settled up.